In [1]:
# import required library
# read dataset 

import pandas as pd
df = pd.read_csv('transactions.csv',sep='|')
df

,sessionID,itemID,click,basket,order
0,0,21310,1,0,0
1,1,73018,1,0,0
2,2,19194,1,0,0
3,3,40250,1,0,0
4,4,46107,1,0,0
...,...,...,...,...,...
365138,279351,70183,1,0,0
365139,279352,39716,1,0,0
365140,279353,35260,1,0,0
365141,279353,18805,4,0,0


In [2]:
# cek missing value

df.isna().sum()

sessionID    0
itemID       0
click        0
basket       0
order        0
dtype: int64

In [3]:
# cek missing value

df[(df['click'] == 0) & (df['basket'] == 0) & (df['order'] == 0)]

,sessionID,itemID,click,basket,order


<h2>Preprocessing dan pemodelan berdasarkan kolom Order</h2>

In [4]:
# memisahkan dataset untuk model rekomendasi berdasarkan order
order = df[['sessionID','itemID','order']]

# filter data tersebut yang memiliki nilai order lebih dari sama dengan 1
ordered = order[order['order'] >= 1]
ordered

,sessionID,itemID,order
21,15,73865,1
66,54,10666,1
74,60,809,1
115,90,38931,1
129,104,74094,1
...,...,...,...
364944,279199,65093,1
364945,279200,43441,1
364983,279238,71270,1
365038,279285,67753,1


In [5]:
# pembuatan dataframe rujukan untuk penyaringan data multitransaksi

sessionID_counts = ordered[['sessionID']].value_counts().sort_index(ascending=True)
index = sessionID_counts.index.tolist()
value = sessionID_counts.tolist()
for_lookup = pd.DataFrame(index,columns=['sessionID'])
for_lookup['sessionID_counts'] = value
for_lookup

,sessionID,sessionID_counts
0,15,1
1,54,1
2,60,1
3,90,1
4,104,3
...,...,...
13240,279199,6
13241,279200,1
13242,279238,1
13243,279285,1


In [6]:
# merge data untuk mendapatkan kuantitas sessionID

ordered = pd.merge(ordered,for_lookup,on='sessionID',how='right')
ordered

,sessionID,itemID,order,sessionID_counts
0,15,73865,1,1
1,54,10666,1,1
2,60,809,1,1
3,90,38931,1,1
4,104,74094,1,3
...,...,...,...,...
16901,279199,65093,1,6
16902,279200,43441,1,1
16903,279238,71270,1,1
16904,279285,67753,1,1


In [8]:
# saring data yang memiliki nilai sessionID_counts lebih dari 1

ordered = ordered[ordered['sessionID_counts'] > 1]
ordered = ordered.drop('sessionID_counts',1)
ordered

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/3264824136.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ordered = ordered.drop('sessionID_counts',1)


,sessionID,itemID,order
4,104,74094,1
5,104,9865,1
6,104,26296,1
12,194,67241,1
13,194,38131,1
...,...,...,...
16897,279199,10039,1
16898,279199,20572,1
16899,279199,58064,1
16900,279199,48821,1


In [9]:
# transformasi dataset menjadi pivot table antara sessionID dan itemID

ordered_transformed = ordered.groupby(['sessionID', 'itemID'])['order'].sum().unstack().reset_index().fillna(0).set_index('sessionID')
ordered_transformed

itemID,54,68,125,194,197,245,249,283,366,367,...,78860,78864,78885,78893,78916,78961,78966,78996,79044,79066
sessionID,,,,,,,,,,,,,,,,,,,,,
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# fungsi untuk mengisi nilai pada pivot table, 0 jika item tidak ada di sessionID, 1 jika ada .
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

# apply fungsi kedalam dataset
ordered_transformed_sets = ordered_transformed.applymap(encode_units)


In [11]:
# Analisis asosiasi menggunakan algoritma apriori
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# menggenerasi frequent itemset dengan nilai min_support = 0.001
frequent_itemsets = apriori(ordered_transformed_sets, min_support=0.001, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.001394,(125)
1,0.001394,(194)
2,0.001859,(245)
3,0.001394,(616)
4,0.001394,(685)
...,...,...
1030,0.001394,"(27041, 14093, 60430, 69073, 47221)"
1031,0.001394,"(27041, 60430, 69073, 47221, 26719)"
1032,0.001394,"(8960, 19458, 36098, 4626, 61335, 48856)"
1033,0.001859,"(8960, 36098, 23654, 4626, 61335, 48856)"


In [12]:
# pembuatan rules dengan minimum confidence = 0.1

rules = association_rules(frequent_itemsets, min_threshold=0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(125),(48678),0.001394,0.001394,0.001394,1.000000,717.333333,0.001392,inf
1,(48678),(125),0.001394,0.001394,0.001394,1.000000,717.333333,0.001392,inf
2,(29485),(782),0.001859,0.001859,0.001394,0.750000,403.500000,0.001391,3.992565
3,(782),(29485),0.001859,0.001859,0.001394,0.750000,403.500000,0.001391,3.992565
4,(41582),(782),0.001859,0.001859,0.001859,1.000000,538.000000,0.001855,inf
...,...,...,...,...,...,...,...,...,...
2955,(14093),"(27041, 60430, 69073, 47221, 26719)",0.007435,0.001394,0.001394,0.187500,134.500000,0.001384,1.229053
2956,(60430),"(27041, 14093, 69073, 47221, 26719)",0.006970,0.001859,0.001394,0.200000,107.600000,0.001381,1.247677
2957,(69073),"(27041, 14093, 60430, 47221, 26719)",0.006506,0.001394,0.001394,0.214286,153.714286,0.001385,1.270953
2958,(47221),"(27041, 14093, 60430, 69073, 26719)",0.007900,0.001394,0.001394,0.176471,126.588235,0.001383,1.212593


In [13]:
# pembuatan array untuk menghitung panjang consequent

consequent_count = []
for i in rules['consequents']:
    consequent_count.append(len(i))

In [14]:
# penyaringan data yang memiliki nilai consequent = 5

rules['consequent_count'] = consequent_count
rules = rules[rules['consequent_count']  == 5 ]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequent_count
2830,(8960),"(19458, 36098, 4626, 61335, 48856)",0.009758,0.001859,0.001394,0.142857,76.857143,0.001376,1.164498,5
2831,(19458),"(8960, 36098, 4626, 61335, 48856)",0.009294,0.002323,0.001394,0.150000,64.560000,0.001372,1.173737,5
2832,(36098),"(8960, 19458, 4626, 61335, 48856)",0.008829,0.001394,0.001394,0.157895,113.263158,0.001382,1.185845,5
2833,(4626),"(8960, 19458, 36098, 61335, 48856)",0.008364,0.001394,0.001394,0.166667,119.555556,0.001382,1.198327,5
2834,(61335),"(8960, 19458, 36098, 4626, 48856)",0.011152,0.001394,0.001394,0.125000,89.666667,0.001379,1.141264,5
2835,(48856),"(8960, 19458, 36098, 4626, 61335)",0.009294,0.001394,0.001394,0.150000,107.600000,0.001381,1.174831,5
2892,(8960),"(36098, 23654, 4626, 61335, 48856)",0.009758,0.001859,0.001859,0.190476,102.476190,0.001841,1.232998,5
2893,(36098),"(8960, 23654, 4626, 61335, 48856)",0.008829,0.001859,0.001859,0.210526,113.263158,0.001842,1.264312,5
2894,(23654),"(8960, 36098, 4626, 61335, 48856)",0.009758,0.002323,0.001859,0.190476,81.980952,0.001836,1.232424,5
2895,(4626),"(8960, 36098, 23654, 61335, 48856)",0.008364,0.001859,0.001859,0.222222,119.555556,0.001843,1.283324,5


In [15]:
# urutkan data berdasarkan lift 
rules = rules.sort_values(by='lift',ascending=False)

# hapus duplikasi antecedent dengan parameter keep='first'
rules.drop_duplicates('antecedents',inplace=True)

# drop unused column
rec_by_order = rules[['antecedents','consequents']]
rec_by_order


,antecedents,consequents
2957,(69073),"(27041, 14093, 60430, 47221, 26719)"
2959,(26719),"(27041, 14093, 60430, 69073, 47221)"
2954,(27041),"(14093, 60430, 69073, 47221, 26719)"
2955,(14093),"(27041, 60430, 69073, 47221, 26719)"
2958,(47221),"(27041, 14093, 60430, 69073, 26719)"
2895,(4626),"(8960, 36098, 23654, 61335, 48856)"
2832,(36098),"(8960, 19458, 4626, 61335, 48856)"
2897,(48856),"(8960, 36098, 23654, 4626, 61335)"
2956,(60430),"(27041, 14093, 69073, 47221, 26719)"
2892,(8960),"(36098, 23654, 4626, 61335, 48856)"


dataset rec_by_order di export kedalam bentuk .csv, dan dimanipulasi menggunakan excel sehingga menjadi seperti dibawah ini

In [16]:
# hasil manipulasi excel

rec_by_order = pd.read_csv('rec_by_order.csv',sep=";")
rec_by_order = rec_by_order.rename(columns={'antecedents':'itemID','consequents':'recomendations'})
rec_by_order

,itemID,recomendations
0,8960,"19458, 36098, 4626, 61335, 48856"
1,19458,"8960, 36098, 4626, 61335, 48856"
2,36098,"8960, 19458, 4626, 61335, 48856"
3,4626,"8960, 19458, 36098, 61335, 48856"
4,61335,"8960, 19458, 36098, 4626, 48856"
5,48856,"8960, 19458, 36098, 4626, 61335"
6,23654,"8960, 36098, 4626, 61335, 48856"
7,27041,"14093, 60430, 69073, 47221, 26719"
8,14093,"27041, 60430, 69073, 47221, 26719"
9,60430,"27041, 14093, 69073, 47221, 26719"


<h2>Preprocessing dan Pemodelan berdasarkan kolom basket</h2>

In [17]:
# memisahkan dataset untuk model rekomendasi berdasarkan basket
basket = df[['sessionID','itemID','basket']]

# filter data tersebut yang memiliki nilai basket lebih dari sama dengan 1
basketed = basket[basket['basket'] >= 1]
basketed

,sessionID,itemID,basket
7,7,14576,1
8,7,17731,1
14,12,29508,1
22,16,15581,1
34,26,23197,1
...,...,...,...
365092,279325,39626,1
365093,279325,33960,2
365103,279333,8703,1
365106,279336,44220,1


In [18]:
# cek unique value dari itemID pada hasil rekomendasi berdasarkan order

rec_by_order.itemID.unique()

array([ 8960, 19458, 36098,  4626, 61335, 48856, 23654, 27041, 14093,
       60430, 69073, 47221, 26719], dtype=int64)

In [19]:
# saring data yang tidak memiliki itemID sama dengan itemID pada hasil rekomendasi berdasarkan order

basketed = basketed[(basketed.itemID != 23654) & (basketed.itemID != 8960) & 
(basketed.itemID != 4626) & (basketed.itemID != 19458) & (basketed.itemID != 36098) & 
(basketed.itemID != 61335) & (basketed.itemID != 48856) &(basketed.itemID != 27041) &
(basketed.itemID != 14093) & (basketed.itemID != 60430) & (basketed.itemID != 69073) &
(basketed.itemID != 47221) & (basketed.itemID != 26719)]

basketed

,sessionID,itemID,basket
7,7,14576,1
8,7,17731,1
14,12,29508,1
22,16,15581,1
34,26,23197,1
...,...,...,...
365092,279325,39626,1
365093,279325,33960,2
365103,279333,8703,1
365106,279336,44220,1


In [20]:
# pembuatan dataframe rujukan untuk penyaringan data multitransaksi

value_counts = basketed[['sessionID']].value_counts().sort_index(ascending=True)
index = value_counts.index.tolist()
value = value_counts.tolist()
for_lookup = pd.DataFrame(index,columns=['sessionID'])
for_lookup['value_counts'] = value
for_lookup

,sessionID,value_counts
0,7,2
1,12,1
2,16,1
3,26,1
4,28,2
...,...,...
29056,279324,4
29057,279325,2
29058,279333,1
29059,279336,1


In [21]:
# merge data untuk mendapatkan kuantitas sessionID

basketed = pd.merge(basketed,for_lookup,on='sessionID',how='right')
basketed

,sessionID,itemID,basket,value_counts
0,7,14576,1,2
1,7,17731,1,2
2,12,29508,1,1
3,16,15581,1,1
4,26,23197,1,1
...,...,...,...,...
43336,279325,39626,1,2
43337,279325,33960,2,2
43338,279333,8703,1,1
43339,279336,44220,1,1


In [22]:
# saring data yang memiliki nilai sessionID_counts lebih dari 1

basketed = basketed[basketed['value_counts'] > 1]
basketed = basketed.drop('value_counts',1)
basketed

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/1157843867.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  basketed = basketed.drop('value_counts',1)


,sessionID,itemID,basket
0,7,14576,1
1,7,17731,1
5,28,57800,1
6,28,11525,1
8,52,65865,1
...,...,...,...
43333,279324,19324,1
43334,279324,78761,1
43335,279324,19070,1
43336,279325,39626,1


In [23]:
# transformasi dataset menjadi pivot table antara sessionID dan itemID

basketed_transformed = basketed.groupby(['sessionID', 'itemID'])['basket'].sum().unstack().reset_index().fillna(0).set_index('sessionID')
basketed_transformed

itemID,13,17,77,110,125,142,158,194,201,239,...,78935,78945,78963,78982,78996,79014,79044,79051,79057,79066
sessionID,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
279261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
279310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# apply fungsi encode kedalam dataset

basketed_transformed_sets = basketed_transformed.applymap(encode_units)

In [25]:
# menggenerasi frequent itemset dengan nilai min_support = 0.0013
 
frequent_itemsets = apriori(basketed_transformed_sets, min_support=0.0013, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.001402,(283)
1,0.001823,(616)
2,0.001542,(782)
3,0.004627,(1193)
4,0.005889,(1607)
...,...,...
892,0.001402,"(62913, 58851, 11525, 1607, 57800, 74654, 14207)"
893,0.001402,"(62913, 58851, 11525, 1607, 57800, 42353, 74654)"
894,0.001402,"(62913, 58851, 1607, 57800, 42353, 74654, 14207)"
895,0.001402,"(62913, 58851, 11525, 57800, 42353, 74654, 14207)"


In [26]:
# pembuatan rules dengan minimum confidence = 0.1

rules = association_rules(frequent_itemsets, min_threshold=0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(29485),(782),0.001823,0.001542,0.001402,0.769231,498.741259,0.001399,4.326650
1,(782),(29485),0.001542,0.001823,0.001402,0.909091,498.741259,0.001399,10.979950
2,(1193),(74094),0.004627,0.006450,0.001823,0.393939,61.077734,0.001793,1.639358
3,(74094),(1193),0.006450,0.004627,0.001823,0.282609,61.077734,0.001793,1.387490
4,(11525),(1607),0.005048,0.005889,0.001963,0.388889,66.037037,0.001933,1.626727
...,...,...,...,...,...,...,...,...,...
7031,(1607),"(62913, 58851, 11525, 57800, 42353, 74654, 14207)",0.005889,0.001402,0.001402,0.238095,169.809524,0.001394,1.310660
7032,(57800),"(62913, 58851, 11525, 1607, 42353, 74654, 14207)",0.004487,0.001402,0.001402,0.312500,222.875000,0.001396,1.452506
7033,(42353),"(62913, 58851, 11525, 1607, 57800, 74654, 14207)",0.005889,0.001402,0.001402,0.238095,169.809524,0.001394,1.310660
7034,(74654),"(62913, 58851, 11525, 1607, 57800, 42353, 14207)",0.004627,0.001402,0.001402,0.303030,216.121212,0.001396,1.432771


In [27]:
# pembuatan array untuk menghitung panjang consequent
consequent_count = []
for i in rules['consequents']:
    consequent_count.append(len(i))

# penyaringan data yang memiliki nilai panjang consequent = 5
rules['consequent_count'] = consequent_count
rules = rules[rules['consequent_count']  == 5 ]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequent_count
4094,(58851),"(11525, 1607, 57800, 42353, 14207)",0.005468,0.001542,0.001402,0.256410,166.247086,0.001394,1.342753,5
4095,(11525),"(58851, 1607, 57800, 42353, 14207)",0.005048,0.001402,0.001402,0.277778,198.111111,0.001395,1.382674,5
4096,(1607),"(58851, 11525, 57800, 42353, 14207)",0.005889,0.001402,0.001402,0.238095,169.809524,0.001394,1.310660,5
4097,(57800),"(58851, 11525, 1607, 42353, 14207)",0.004487,0.001402,0.001402,0.312500,222.875000,0.001396,1.452506,5
4098,(42353),"(58851, 11525, 1607, 57800, 14207)",0.005889,0.001402,0.001402,0.238095,169.809524,0.001394,1.310660,5
...,...,...,...,...,...,...,...,...,...,...
6995,"(14207, 74654, 1607)","(62913, 58851, 11525, 57800, 42353)",0.001402,0.001402,0.001402,1.000000,713.200000,0.001400,inf,5
6996,"(57800, 42353, 74654)","(62913, 58851, 11525, 1607, 14207)",0.001542,0.001402,0.001402,0.909091,648.363636,0.001400,10.984577,5
6997,"(57800, 42353, 14207)","(62913, 58851, 11525, 1607, 74654)",0.001683,0.001542,0.001402,0.833333,540.303030,0.001400,5.990746,5
6998,"(57800, 74654, 14207)","(62913, 58851, 11525, 1607, 42353)",0.001542,0.001402,0.001402,0.909091,648.363636,0.001400,10.984577,5


dari rules yang dihasilkan berdasarkan basket, terdapat panjang antecedents yang lebih dari 1, sehingga kita perlu memfilter antecedents yang hanya sama dengan 1

In [28]:
# pembuatan array untuk menghitung panjang antecedents
antecedents_count = []
for i in rules['antecedents']:
    antecedents_count.append(len(i))

# penyaringan data yang memiliki panjang antecedents = 1
rules['antecedents_count'] = antecedents_count
rules = rules[rules['antecedents_count']  == 1 ]
rules

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/1573245528.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rules['antecedents_count'] = antecedents_count


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequent_count,antecedents_count
4094,(58851),"(11525, 1607, 57800, 42353, 14207)",0.005468,0.001542,0.001402,0.256410,166.247086,0.001394,1.342753,5,1
4095,(11525),"(58851, 1607, 57800, 42353, 14207)",0.005048,0.001402,0.001402,0.277778,198.111111,0.001395,1.382674,5,1
4096,(1607),"(58851, 11525, 57800, 42353, 14207)",0.005889,0.001402,0.001402,0.238095,169.809524,0.001394,1.310660,5,1
4097,(57800),"(58851, 11525, 1607, 42353, 14207)",0.004487,0.001402,0.001402,0.312500,222.875000,0.001396,1.452506,5,1
4098,(42353),"(58851, 11525, 1607, 57800, 14207)",0.005889,0.001402,0.001402,0.238095,169.809524,0.001394,1.310660,5,1
...,...,...,...,...,...,...,...,...,...,...,...
5769,(58851),"(62913, 57800, 42353, 74654, 14207)",0.005468,0.001402,0.001402,0.256410,182.871795,0.001394,1.342942,5,1
5770,(57800),"(62913, 58851, 42353, 74654, 14207)",0.004487,0.001402,0.001402,0.312500,222.875000,0.001396,1.452506,5,1
5771,(42353),"(62913, 58851, 57800, 74654, 14207)",0.005889,0.001402,0.001402,0.238095,169.809524,0.001394,1.310660,5,1
5772,(74654),"(62913, 58851, 57800, 42353, 14207)",0.004627,0.001402,0.001402,0.303030,216.121212,0.001396,1.432771,5,1


In [29]:
# urutkan data berdasarkan lift 
rules = rules.sort_values(by='lift',ascending=False)

# hapus duplikasi antecedent dengan parameter keep='first'
rules.drop_duplicates('antecedents',inplace=True)

# drop unused column
rec_by_basket = rules[['antecedents','consequents']]
rec_by_basket

,antecedents,consequents
5770,(57800),"(62913, 58851, 42353, 74654, 14207)"
4347,(14207),"(58851, 11525, 1607, 42353, 74654)"
4594,(74654),"(62913, 11525, 1607, 57800, 14207)"
5398,(11525),"(62913, 58851, 57800, 42353, 14207)"
5335,(58851),"(62913, 1607, 57800, 42353, 74654)"
5396,(62913),"(58851, 11525, 57800, 42353, 14207)"
4406,(1607),"(62913, 11525, 42353, 74654, 14207)"
4719,(42353),"(62913, 58851, 11525, 1607, 57800)"


dataset rec_by_basket di export kedalam bentuk .csv, dan dimanipulasi menggunakan excel sehingga menjadi seperti dibawah ini

In [30]:
# hasil manipulasi excel

rec_by_basket = pd.read_csv('rec_by_basket.csv',sep=';')
rec_by_basket = rec_by_basket.rename(columns={'antecedents':'itemID','consequents':'recomendations'})
rec_by_basket

,itemID,recomendations
0,57800,"62913, 58851, 42353, 74654, 14207"
1,14207,"58851, 11525, 1607, 42353, 74654"
2,74654,"62913, 11525, 1607, 57800, 14207"
3,11525,"62913, 58851, 57800, 42353, 14207"
4,58851,"62913, 1607, 57800, 42353, 74654"
5,62913,"58851, 11525, 57800, 42353, 14207"
6,1607,"62913, 11525, 42353, 74654, 14207"
7,42353,"62913, 58851, 11525, 1607, 57800"


<h2>Preprocessing dan Pemodelan berdasarkan kolom klik</h2>

In [31]:
# memisahkan dataset untuk model rekomendasi berdasarkan click
click = df[['sessionID','itemID','click']]

# filter data tersebut yang memiliki nilai click lebih dari sama dengan 1
clicked = click[click['click'] >= 1]
clicked

,sessionID,itemID,click
0,0,21310,1
1,1,73018,1
2,2,19194,1
3,3,40250,1
4,4,46107,1
...,...,...,...
365138,279351,70183,1
365139,279352,39716,1
365140,279353,35260,1
365141,279353,18805,4


In [32]:
# cek unique value dari itemID pada hasil rekomendasi berdasarkan order dan basket

print(rec_by_order['itemID'].unique())
print(rec_by_basket['itemID'].unique())

[ 8960 19458 36098  4626 61335 48856 23654 27041 14093 60430 69073 47221
 26719]
[57800 14207 74654 11525 58851 62913  1607 42353]


In [33]:
# saring data yang tidak memiliki itemID sama dengan itemID pada hasil rekomendasi berdasarkan order dan basket

clicked = clicked[(clicked.itemID != 23654) & (clicked.itemID != 8960) & 
(clicked.itemID != 4626) & (clicked.itemID != 19458) & (clicked.itemID != 36098) & 
(clicked.itemID != 61335) & (clicked.itemID != 48856) &(clicked.itemID != 27041) &
(clicked.itemID != 14093) & (clicked.itemID != 60430) & (clicked.itemID != 69073) &
(clicked.itemID != 47221) & (clicked.itemID != 26719) & (clicked.itemID != 62913) & 
(clicked.itemID != 58851) & (clicked.itemID != 1607) & (clicked.itemID != 57800) & 
(clicked.itemID != 42353) & (clicked.itemID != 74654)]

clicked

,sessionID,itemID,click
0,0,21310,1
1,1,73018,1
2,2,19194,1
3,3,40250,1
4,4,46107,1
...,...,...,...
365138,279351,70183,1
365139,279352,39716,1
365140,279353,35260,1
365141,279353,18805,4


In [34]:
# pembuatan dataframe rujukan untuk penyaringan data multitransaksi

value_counts = clicked[['sessionID']].value_counts().sort_index(ascending=True)
index = value_counts.index.tolist()
value = value_counts.tolist()
for_lookup = pd.DataFrame(index,columns=['sessionID'])
for_lookup['value_counts'] = value
for_lookup

,sessionID,value_counts
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
254624,279350,4
254625,279351,1
254626,279352,1
254627,279353,2


In [35]:
# merge data untuk mendapatkan kuantitas sessionID

clicked = pd.merge(clicked,for_lookup,on='sessionID',how='right')
clicked

,sessionID,itemID,click,value_counts
0,0,21310,1,1
1,1,73018,1,1
2,2,19194,1,1
3,3,40250,1,1
4,4,46107,1,1
...,...,...,...,...
336606,279351,70183,1,1
336607,279352,39716,1,1
336608,279353,35260,1,2
336609,279353,18805,4,2


In [36]:
# saring data yang memiliki nilai sessionID_counts lebih dari 1

clicked = clicked[clicked['value_counts'] > 1]
clicked = clicked.drop('value_counts',1)
clicked

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/3347490560.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  clicked = clicked.drop('value_counts',1)


,sessionID,itemID,click
7,7,14576,1
8,7,17731,2
13,12,30277,1
14,12,29508,1
15,12,75659,1
...,...,...,...
336603,279350,73084,1
336604,279350,13996,1
336605,279350,23084,1
336608,279353,35260,1


In [37]:
# transformasi dataset menjadi pivot table antara sessionID dan itemID

clicked_transformed = clicked.groupby(['sessionID', 'itemID'])['click'].sum().unstack().reset_index().fillna(0).set_index('sessionID')
clicked_transformed

itemID,1,2,6,13,17,18,35,43,64,68,...,78996,79014,79038,79043,79044,79051,79057,79058,79062,79066
sessionID,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
279343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
279344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# apply fungsi encode kedalam dataset

clicked_transformed_sets = clicked_transformed.applymap(encode_units)

In [50]:
# menggenerasi frequent itemset dengan nilai minimun support = 0.0022

frequent_itemsets = apriori(clicked_transformed_sets, min_support=0.0022, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.002345,(616)
1,0.002535,(1193)
2,0.004753,(1713)
3,0.011597,(2253)
4,0.004404,(2417)
...,...,...
241,0.002313,"(5226, 50523, 12723)"
242,0.003137,"(32266, 5291, 29711)"
243,0.002471,"(25451, 72468, 31995)"
244,0.002250,"(33908, 25451, 72468)"


In [51]:
# pembuatan rules dengan minimum confidence = 0.1

rules = association_rules(frequent_itemsets, min_threshold=0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1713),(36071),0.004753,0.007890,0.003200,0.673333,85.342972,0.003163,3.037072
1,(36071),(1713),0.007890,0.004753,0.003200,0.405622,85.342972,0.003163,1.674436
2,(2253),(11743),0.011597,0.007129,0.004753,0.409836,57.486339,0.004670,1.682364
3,(11743),(2253),0.007129,0.011597,0.004753,0.666667,57.486339,0.004670,2.965209
4,(2253),(17727),0.011597,0.012833,0.002376,0.204918,15.968427,0.002228,1.241592
...,...,...,...,...,...,...,...,...,...
107,"(33908, 72468)",(31995),0.002757,0.006812,0.002281,0.827586,121.481957,0.002263,5.760488
108,"(31995, 72468)",(33908),0.003802,0.003739,0.002281,0.600000,160.474576,0.002267,2.490653
109,(33908),"(31995, 72468)",0.003739,0.003802,0.002281,0.610169,160.474576,0.002267,2.555464
110,(31995),"(33908, 72468)",0.006812,0.002757,0.002281,0.334884,121.481957,0.002263,1.499352


In [52]:
# pembuatan array untuk menghitung panjang consequent
consequent_count = []
for i in rules['consequents']:
    consequent_count.append(len(i))

# penyaringan data yang memiliki nilai panjang consequent >= 2
rules['consequent_count'] = consequent_count
rules = rules[rules['consequent_count']  >= 2 ]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequent_count
79,(2253),"(46998, 11743)",0.011597,0.004499,0.003485,0.300546,66.797506,0.003433,1.423255,2
80,(46998),"(2253, 11743)",0.019740,0.004753,0.003485,0.176565,37.149278,0.003392,1.208653,2
81,(11743),"(2253, 46998)",0.007129,0.006971,0.003485,0.488889,70.133333,0.003436,1.942883,2
85,(5226),"(50523, 12723)",0.011185,0.002757,0.002313,0.206799,75.018072,0.002282,1.257239,2
86,(50523),"(5226, 12723)",0.005672,0.003422,0.002313,0.407821,119.174426,0.002294,1.682900,2
87,(12723),"(5226, 50523)",0.004594,0.003644,0.002313,0.503448,138.163718,0.002296,2.006551,2
91,(32266),"(5291, 29711)",0.007224,0.003897,0.003137,0.434211,111.412067,0.003109,1.760554,2
92,(5291),"(32266, 29711)",0.012199,0.003707,0.003137,0.257143,69.362637,0.003092,1.341163,2
93,(29711),"(32266, 5291)",0.005323,0.004404,0.003137,0.589286,133.797533,0.003113,2.424059,2
97,(25451),"(31995, 72468)",0.003992,0.003802,0.002471,0.619048,162.809524,0.002456,2.615019,2


In [53]:
# urutkan data berdasarkan lift 
rules = rules.sort_values(by='lift',ascending=False)

# hapus duplikasi antecedent dengan parameter keep='first'
rules.drop_duplicates('antecedents',inplace=True)

# drop unused column
rec_by_click = rules[['antecedents','consequents']]
rec_by_click

,antecedents,consequents
104,(25451),"(33908, 72468)"
103,(33908),"(25451, 72468)"
105,(72468),"(25451, 33908)"
87,(12723),"(5226, 50523)"
93,(29711),"(32266, 5291)"
110,(31995),"(33908, 72468)"
86,(50523),"(5226, 12723)"
91,(32266),"(5291, 29711)"
85,(5226),"(50523, 12723)"
81,(11743),"(2253, 46998)"


dataset rec_by_click di export kedalam bentuk .csv, dan dimanipulasi menggunakan excel sehingga menjadi seperti dibawah ini

In [54]:
# hasil manipulasi excel

rec_by_click = pd.read_csv('rec_by_click.csv',sep=";")
rec_by_click = rec_by_click.rename(columns={'antecedents':'itemID','consequents':'recomendations'})
rec_by_click

,itemID,recomendations
0,25451,"33908, 72468"
1,33908,"25451, 72468"
2,72468,"25451, 33908"
3,12723,"5226, 50523"
4,29711,"32266, 5291"
5,31995,"25451, 33908"
6,50523,"5226, 12723"
7,32266,"5291, 29711"
8,5226,"50523, 12723"
9,11743,"2253, 46998"


<h2>Hasil</h2>

In [55]:
# menggabungkan hasil rekomendasi dari ketiga bagian

recomendations = pd.concat([rec_by_order,rec_by_basket,rec_by_click])
recomendations = recomendations.reset_index()
recomendations = recomendations.drop('index',1)
recomendations

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/1161810740.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  recomendations = recomendations.drop('index',1)


,itemID,recomendations
0,8960,"19458, 36098, 4626, 61335, 48856"
1,19458,"8960, 36098, 4626, 61335, 48856"
2,36098,"8960, 19458, 4626, 61335, 48856"
3,4626,"8960, 19458, 36098, 61335, 48856"
4,61335,"8960, 19458, 36098, 4626, 48856"
5,48856,"8960, 19458, 36098, 4626, 61335"
6,23654,"8960, 36098, 4626, 61335, 48856"
7,27041,"14093, 60430, 69073, 47221, 26719"
8,14093,"27041, 60430, 69073, 47221, 26719"
9,60430,"27041, 14093, 69073, 47221, 26719"


In [56]:
# proses manipulasi dataframe

split1 = recomendations.head(21)
split2 = recomendations.tail(13)

In [57]:
# proses manipulasi dataframe

data1 = split1['recomendations'].str.split(",", n = 4, expand = True)
data2 = split2['recomendations'].str.split(",", n = 1, expand = True)

In [58]:
# proses manipulasi dataframe

split1['rec1'] = data1[0]
split1['rec2'] = data1[1]
split1['rec3'] = data1[2]
split1['rec4'] = data1[3]
split1['rec5'] = data1[4]
split1

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/188325872.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split1['rec1'] = data1[0]
C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/188325872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split1['rec2'] = data1[1]
C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/188325872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,itemID,recomendations,rec1,rec2,rec3,rec4,rec5
0,8960,"19458, 36098, 4626, 61335, 48856",19458,36098,4626,61335,48856
1,19458,"8960, 36098, 4626, 61335, 48856",8960,36098,4626,61335,48856
2,36098,"8960, 19458, 4626, 61335, 48856",8960,19458,4626,61335,48856
3,4626,"8960, 19458, 36098, 61335, 48856",8960,19458,36098,61335,48856
4,61335,"8960, 19458, 36098, 4626, 48856",8960,19458,36098,4626,48856
5,48856,"8960, 19458, 36098, 4626, 61335",8960,19458,36098,4626,61335
6,23654,"8960, 36098, 4626, 61335, 48856",8960,36098,4626,61335,48856
7,27041,"14093, 60430, 69073, 47221, 26719",14093,60430,69073,47221,26719
8,14093,"27041, 60430, 69073, 47221, 26719",27041,60430,69073,47221,26719
9,60430,"27041, 14093, 69073, 47221, 26719",27041,14093,69073,47221,26719


In [59]:
# proses manipulasi dataframe

split2['rec1'] = data2[0]
split2['rec2'] = data2[1]
split2

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/2128835616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split2['rec1'] = data2[0]
C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/2128835616.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split2['rec2'] = data2[1]


,itemID,recomendations,rec1,rec2
21,25451,"33908, 72468",33908,72468
22,33908,"25451, 72468",25451,72468
23,72468,"25451, 33908",25451,33908
24,12723,"5226, 50523",5226,50523
25,29711,"32266, 5291",32266,5291
26,31995,"25451, 33908",25451,33908
27,50523,"5226, 12723",5226,12723
28,32266,"5291, 29711",5291,29711
29,5226,"50523, 12723",50523,12723
30,11743,"2253, 46998",2253,46998


In [60]:
# hasil manipulasi

recomendations = pd.concat([split1,split2])
recomendations = recomendations.reset_index()
recomendations = recomendations.drop(['index','recomendations'],1)
recomendations

C:\Users\Muhith\AppData\Local\Temp/ipykernel_28540/3121410330.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  recomendations = recomendations.drop(['index','recomendations'],1)


,itemID,rec1,rec2,rec3,rec4,rec5
0,8960,19458,36098,4626,61335,48856
1,19458,8960,36098,4626,61335,48856
2,36098,8960,19458,4626,61335,48856
3,4626,8960,19458,36098,61335,48856
4,61335,8960,19458,36098,4626,48856
5,48856,8960,19458,36098,4626,61335
6,23654,8960,36098,4626,61335,48856
7,27041,14093,60430,69073,47221,26719
8,14093,27041,60430,69073,47221,26719
9,60430,27041,14093,69073,47221,26719


data hasil manipulasi (recomendations) di export kedalam bentuk .csv, dan dilakukan vlookup menggunakan excel sehingga menjadi seperti dibawah ini

In [61]:
# hasil akhir

recomendations = pd.read_csv('recomendations.csv',sep=";")
recomendations

,Items,rec1,rec2,rec3,rec4,rec5
0,Harry Potter 2 und die Kammer des Schreckens,Harry Potter 4 und der Feuerkelch. Taschenbuch,Harry Potter 6 und der Halbblutprinz,Harry Potter 5 und der Orden des Phönix,Harry Potter 3 und der Gefangene von Askaban,Harry Potter 7 und die Heiligtümer des Todes
1,Harry Potter 4 und der Feuerkelch. Taschenbuch,Harry Potter 2 und die Kammer des Schreckens,Harry Potter 6 und der Halbblutprinz,Harry Potter 5 und der Orden des Phönix,Harry Potter 3 und der Gefangene von Askaban,Harry Potter 7 und die Heiligtümer des Todes
2,Harry Potter 6 und der Halbblutprinz,Harry Potter 2 und die Kammer des Schreckens,Harry Potter 4 und der Feuerkelch. Taschenbuch,Harry Potter 5 und der Orden des Phönix,Harry Potter 3 und der Gefangene von Askaban,Harry Potter 7 und die Heiligtümer des Todes
3,Harry Potter 5 und der Orden des Phönix,Harry Potter 2 und die Kammer des Schreckens,Harry Potter 4 und der Feuerkelch. Taschenbuch,Harry Potter 6 und der Halbblutprinz,Harry Potter 3 und der Gefangene von Askaban,Harry Potter 7 und die Heiligtümer des Todes
4,Harry Potter 3 und der Gefangene von Askaban,Harry Potter 2 und die Kammer des Schreckens,Harry Potter 4 und der Feuerkelch. Taschenbuch,Harry Potter 6 und der Halbblutprinz,Harry Potter 5 und der Orden des Phönix,Harry Potter 7 und die Heiligtümer des Todes
5,Harry Potter 7 und die Heiligtümer des Todes,Harry Potter 2 und die Kammer des Schreckens,Harry Potter 4 und der Feuerkelch. Taschenbuch,Harry Potter 6 und der Halbblutprinz,Harry Potter 5 und der Orden des Phönix,Harry Potter 3 und der Gefangene von Askaban
6,Harry Potter 1 und der Stein der Weisen,Harry Potter 2 und die Kammer des Schreckens,Harry Potter 6 und der Halbblutprinz,Harry Potter 5 und der Orden des Phönix,Harry Potter 3 und der Gefangene von Askaban,Harry Potter 7 und die Heiligtümer des Todes
7,Throne of Glass 5 - Die Sturmbezwingerin,Throne of Glass 06 - Der verwundete Krieger,Throne of Glass 2 - Kriegerin im Schatten,Throne of Glass 4 - Königin der Finsternis,Throne of Glass 7 - Herrscherin über Asche und...,Throne of Glass 3 - Erbin des Feuers
8,Throne of Glass 06 - Der verwundete Krieger,Throne of Glass 5 - Die Sturmbezwingerin,Throne of Glass 2 - Kriegerin im Schatten,Throne of Glass 4 - Königin der Finsternis,Throne of Glass 7 - Herrscherin über Asche und...,Throne of Glass 3 - Erbin des Feuers
9,Throne of Glass 2 - Kriegerin im Schatten,Throne of Glass 5 - Die Sturmbezwingerin,Throne of Glass 06 - Der verwundete Krieger,Throne of Glass 4 - Königin der Finsternis,Throne of Glass 7 - Herrscherin über Asche und...,Throne of Glass 3 - Erbin des Feuers
